In [58]:
import datetime
import os
import random
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy
import statsmodels.api as sm
import pylab as py

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.linear_model import LinearRegression

import math
import time
import datetime
from dython.nominal import associations
from dython.nominal import identify_nominal_columns

In [59]:
data = pd.read_csv("../../dataset/final_bike_sharing.csv")

In [60]:
cols = ['tripduration','gender','dist','hour','birthyear','usertype']

In [61]:
df = data[cols]

## Before Imputation the distributaion is 

In [62]:
# for col in  ['tripduration','gender','dist','hour','birthyear']:
#     np.random.seed(19680801)
#     plt.figure(figsize=(20,8))
#     mu = df[col].mean()
#     sigma = df[col].std()
#     x = df[col]
#     num_bins = 500
#     fig, ax = plt.subplots()
#     # the histogram of the data
#     n, bins, patches = ax.hist(x, num_bins, density=True)
#     # add a 'best fit' line
#     y = ((1 / (np.sqrt(2 * np.pi) * sigma)) *
#          np.exp(-0.5 * (1 / sigma * (bins - mu))**2))
#     ax.plot(bins, y, '--')
#     ax.set_xlabel(col)
#     ax.set_ylabel('Probability density')
#     ax.set_title(f'Histogram of IQ: {mu}, {sigma}')
#     plt.plot([mu,mu],[0, 2], lw=1.8, label="mean")
# #     plt.plot([sigma,sigma],[0, 2], lw=1.8, label="std")
# #     ax.set_ylim([0,2.5])
# #     ax.set_xlim([0, 6])
#     # Tweak spacing to prevent clipping of ylabel
#     fig.tight_layout()
#     plt.show()

In [63]:
# for col in df.columns:
#     norm_cdf = scipy.stats.norm.cdf(df[col]) # calculate the cdf - also discrete
#     # plot the cdf
#     sns.lineplot(x=df[col], y=norm_cdf)
#     plt.show()
#     sm.qqplot(df[col], line ='45')
#     py.show()


In [64]:
for col in  ['tripduration','dist','hour','birthyear']:
    mu = df[col].mean()
    sigma = df[col].std()
    print(f"The variable {col} has mean = {mu} and standard deviation ={sigma} ")
print(f"The variable usertype categorical type mode is ", df['usertype'].mode() )  
print("gender mode is : " , df['gender'].mode() )

The variable tripduration has mean = 525.9314525266259 and standard deviation =731.0970649533257 
The variable dist has mean = 0.7842291293398377 and standard deviation =0.5421989613484405 
The variable hour has mean = 13.633500453206436 and standard deviation =5.156725828688158 
The variable birthyear has mean = 1980.439108882846 and standard deviation =10.094119857977768 
The variable usertype categorical type mode is  0    Subscriber
Name: usertype, dtype: object
gender mode is :  0    male
Name: gender, dtype: object


In [65]:
def split_dataset(df):
    X = df.drop('tripduration', axis=1)
    y = df['tripduration']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print('Train', X_train.shape, y_train.shape)
    print('Test', X_test.shape, y_test.shape)
    return X_train, X_test, y_train, y_test

In [66]:
## get the sample size such as 5%, 10%,20% etc
def get_sample_size(df, perc):
    return round(perc/100 * df.shape[0])

def get_an_index(index):
    li = []
    for i in index:
        li.append(i)
    return li   

def ampute_nan_values(X_train,col_name, perc):
    random.seed(100)
    index = get_an_index(X_train.index)
    sample_size = get_sample_size(X_train, perc)
    selected_index = random.sample(index, sample_size)
    for i in selected_index:
        if col_name in X_train.columns:
            X_train.at[i,col_name]= np.NaN
        else:
            X_train.loc[i] = np.NaN
    return X_train

###====================####
def convert_to_dataframe(y_train):
    return pd.DataFrame(y_train)

def convert_to_series(y_train):
    return y_train.squeeze()

def ampute_nan_values_drop(X_train,y_train,col_name, perc):
    random.seed(100)
    index = get_an_index(X_train.index)
    sample_size = get_sample_size(X_train, perc)
    selected_index = random.sample(index, sample_size)
    y_train = convert_to_dataframe(y_train)
    for i in selected_index:
        if col_name in X_train.columns:
            X_train.at[i,col_name]= np.NaN
        else:
            X_train.loc[i] = np.NaN
        y_train.loc[i] = np.NaN
    y_train = convert_to_series(y_train)
    return X_train, y_train

In [67]:
def sample_imputation_X(var,X_train,X_test):
    # extract a random sample
    random_sample_train = X_train[var].dropna().sample(X_train[var].isnull().sum(), random_state=0)
    random_sample_test = X_test[var].dropna().sample(X_test[var].isnull().sum(), random_state=0)
     # re-index the randomly extracted sample
    random_sample_train.index = X_train[X_train[var].isnull()].index
    random_sample_test.index = X_test[X_test[var].isnull()].index

    # replace the NA
    X_train.loc[X_train[var].isnull(), var] = random_sample_train
    X_test.loc[X_test[var].isnull(), var] = random_sample_test
    return X_train, X_test

In [68]:
def one_hot_encoding(X_train,X_test):
    categorical_features=identify_nominal_columns(X_train)
    print(categorical_features)
    one_hot_encoded_data_train = pd.get_dummies(X_train, columns = categorical_features)
    one_hot_encoded_data_test = pd.get_dummies(X_test, columns = categorical_features)
    return one_hot_encoded_data_train,one_hot_encoded_data_test

In [69]:
def model_evaluation(X_train,y_train):
    print(datetime.datetime.now())
    model_lr = LinearRegression(fit_intercept=True).fit(X_train, y_train) 
    print(datetime.datetime.now())
    return model_lr

In [70]:
def ampute_each_variables(variable, method,X_train,y_train,X_test,y_test):
    if method == "remove":
        X_train , y_train = ampute_nan_values_drop(X_train,y_train,variable,perc=20)
        print('Train set after Amputation', X_train.shape, y_train.shape)
        print("y_train Amputed value count of varaible ", y_train.isnull().sum())
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test , y_test = ampute_nan_values_drop(X_test,y_test,variable,perc=20)
        print('Test set after Amputation', X_test.shape, y_test.shape)
        print("y_test Amputed value count of varaible ", y_test.isnull().sum())
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    else:
        X_train = ampute_nan_values(X_train,variable,perc=20)
        print('Train set after Amputation', X_train.shape)
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test = ampute_nan_values(X_test,variable,perc=20)
        print('Test set after Amputation', X_test.shape)
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    
    if method == "remove":
        print("Simulate Imputation for NA")
        X_train.dropna(axis=0, inplace=True)
        y_train.dropna(inplace=True)
        print('Train after NA value removal Imputation', X_train.shape, y_train.shape)
        X_test.dropna(axis=0, inplace=True)
        y_test.dropna(inplace=True)
        print('Test after NA value removal Imputation', X_test.shape, y_test.shape)
    if method == "mode":
        print("Simulate Imputation for Mode")
        X_train.fillna(X_train.mode().iloc[0], inplace=True)
        print('Train after Mode Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test.mode().iloc[0], inplace=True)
        print('Test after Mode Imputation', X_test.shape, y_test.shape)
    if method == "mean":
        print("Simulate Imputation for Mean")
        X_train.fillna(X_train.mean(), inplace=True)
        print('Train after Mean Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test.mean(), inplace=True)
        print('Test after Mean Imputation', X_test.shape, y_test.shape)
    if method == "sample":
        print("Simple Imputation using sampling")
        X_train, X_test = sample_imputation_X(variable, X_train, X_test)        
            
    return X_train , y_train, X_test , y_test

In [71]:
def visualize_dist(df, col):
    np.random.seed(19680801)
    plt.figure(figsize=(20,8))
    mu = df[col].mean()
    sigma = df[col].std()
    x = df[col]
    num_bins = 500
    fig, ax = plt.subplots()
    # the histogram of the data
    n, bins, patches = ax.hist(x, num_bins, density=True)
    # add a 'best fit' line
    y = ((1 / (np.sqrt(2 * np.pi) * sigma)) *
         np.exp(-0.5 * (1 / sigma * (bins - mu))**2))
    ax.plot(bins, y, '--')
    ax.set_xlabel(col)
    ax.set_ylabel('Probability density')
    ax.set_title(f'Histogram of IQ: {mu}, {sigma}')
    plt.plot([mu,mu],[0, 2], lw=1.8, label="mean")
    #     plt.plot([sigma,sigma],[0, 2], lw=1.8, label="std")
    #     ax.set_ylim([0,2.5])
    #     ax.set_xlim([0, 6])
    # Tweak spacing to prevent clipping of ylabel
    fig.tight_layout()
    plt.show()

In [72]:
features_for_amputation =['gender','dist','hour','birthyear','usertype']

## Calculate mean and std before imputation

In [73]:
X_train, X_test, y_train, y_test = split_dataset(df)

Train (282432, 5) (282432,)
Test (70608, 5) (70608,)


In [74]:
for col in  ['dist','hour','birthyear']:
    mu = X_train[col].mean()
    sigma = X_train[col].std()
    print(f"The variable {col} has mean = {mu} and standard deviation ={sigma} ")
print(f"The variable usertype has mode ",  X_train['usertype'].mode())
print(f"The variable gender categorical type mode is ", X_train['gender'].mode() ) 

The variable dist has mean = 0.7848814720776504 and standard deviation =0.54278813457282 
The variable hour has mean = 13.633274558123725 and standard deviation =5.155597162884377 
The variable birthyear has mean = 1980.450076478586 and standard deviation =10.101242979840942 
The variable usertype has mode  0    Subscriber
Name: usertype, dtype: object
The variable gender categorical type mode is  0    male
Name: gender, dtype: object


In [75]:
for col in ['dist','hour','birthyear']:
    mu = X_test[col].mean()
    sigma = X_test[col].std()
    print(f"The variable {col} has mean = {mu} and standard deviation ={sigma} ")
print(f"The variable usertype has mode ",  X_test['usertype'].mode())
print(f"The variable gender categorical type mode is ", X_test['gender'].mode() ) 

The variable dist has mean = 0.7816197583886165 and standard deviation =0.539831786399696 
The variable hour has mean = 13.634404033537276 and standard deviation =5.161274457822756 
The variable birthyear has mean = 1980.3952384998868 and standard deviation =10.065528836178935 
The variable usertype has mode  0    Subscriber
Name: usertype, dtype: object
The variable gender categorical type mode is  0    male
Name: gender, dtype: object


In [76]:
cat_gender = ['gender_female', 'gender_male','gender_unknown']
cat_usertype = ['usertype_Customer', 'usertype_Subscriber']

## Imputation for NA ( drop NA values)

In [77]:
model_metrics ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"remove",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    if(var == 'gender'):
        for v in cat_gender:
            mu_train = X_train[v].mean()
            sigma_train = X_train[v].std()
            mu_test = X_test[v].mean()
            sigma_test = X_test[v].std()
            model_metrics[v] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                           "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}
    elif(var == 'usertype'):
        for v in cat_usertype:
            mu_train = X_train[v].mean()
            sigma_train = X_train[v].std()
            mu_test = X_test[v].mean()
            sigma_test = X_test[v].std()
            model_metrics[v] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                            "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}
    else:
        mu_train = X_train[var].mean()
        sigma_train = X_train[var].std()
        mu_test = X_test[var].mean()
        sigma_test = X_test[var].std()
        model_metrics[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                              "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                               "intercept":model_reg.intercept_,
                              "Mu_train":mu_train, "sigma_train":sigma_train,
                              "Mu_test":mu_test, "sigma_test":sigma_test}
    del X_train
    del X_test
    del y_train
    del y_test

Now running model is .... gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5) (282432,)
y_train Amputed value count of varaible  56486
X_train Amputed value count of varaible  56486
Test set after Amputation (70608, 5) (70608,)
y_test Amputed value count of varaible  14122
X_test Amputed value count of varaible  14122
Simulate Imputation for NA
Train after NA value removal Imputation (225946, 5) (225946,)
Test after NA value removal Imputation (56486, 5) (56486,)
['gender', 'usertype']
2023-02-19 14:15:18.676519
2023-02-19 14:15:18.759604
Now running model is .... dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5) (282432,)
y_train Amputed value count of varaible  56486
X_train Amputed value count of varaible  56486
Test set after Amputation (70608, 5) (70608,)
y_test Amputed value count of varaible  14122
X_test Amputed value count of varaible  14122
Simulate Imputation for NA
Train after NA v

In [78]:
model_metrics

{'gender_female': {'MAE': 278.73865762235374,
  'MSE': 422975.10446836834,
  'RMSE': 650.3653622913572,
  'R2': 0.16997851844153733,
  'intercept': 4267.144804996973,
  'Mu_train': 0.2129004275357829,
  'sigma_train': 0.40935873894239,
  'Mu_test': 0.20999893778989484,
  'sigma_test': 0.4073110862339015},
 'gender_male': {'MAE': 278.73865762235374,
  'MSE': 422975.10446836834,
  'RMSE': 650.3653622913572,
  'R2': 0.16997851844153733,
  'intercept': 4267.144804996973,
  'Mu_train': 0.7271294911173466,
  'sigma_train': 0.4454358229964737,
  'Mu_test': 0.7307474418439968,
  'sigma_test': 0.4435753615875751},
 'gender_unknown': {'MAE': 278.73865762235374,
  'MSE': 422975.10446836834,
  'RMSE': 650.3653622913572,
  'R2': 0.16997851844153733,
  'intercept': 4267.144804996973,
  'Mu_train': 0.05997008134687049,
  'sigma_train': 0.23743192749051348,
  'Mu_test': 0.059253620366108414,
  'sigma_test': 0.23610085916153653},
 'dist': {'MAE': 278.73865762235374,
  'MSE': 422975.10446836834,
  'RMSE

### Bootstarp with drop NA

In [79]:
bootstrap_data_drop ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"remove",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx = np.arange(y_train.shape[0])
    bootstarp_train_accuracies = {}
    for i in range(501):
        train_idx = rng.choice(idx, size=idx.shape[0], replace=True)
        test_idx = np.setdiff1d(idx, train_idx, assume_unique=False)
        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]
        boot_test_X, boot_test_y = X_train.iloc[test_idx], y_train.iloc[test_idx]
        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_drop[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5) (282432,)
y_train Amputed value count of varaible  56486
X_train Amputed value count of varaible  56486
Test set after Amputation (70608, 5) (70608,)
y_test Amputed value count of varaible  14122
X_test Amputed value count of varaible  14122
Simulate Imputation for NA
Train after NA value removal Imputation (225946, 5) (225946,)
Test after NA value removal Imputation (56486, 5) (56486,)
['gender', 'usertype']
Now running model is .... dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5) (282432,)
y_train Amputed value count of varaible  56486
X_train Amputed value count of varaible  56486
Test set after Amputation (70608, 5) (70608,)
y_test Amputed value count of varaible  14122
X_test Amputed value count of varaible  14122
Simulate Imputation for NA
Train after NA value removal Imputation (225946, 5) (225946,)
Test aft

In [80]:
bootstrap_data_drop

{'gender': {'R2': [0.16943407382491632, 0.18401755037351225],
  'RMSE': [649.3841484106034, 682.964131050179],
  'intercept': [3697.38996191378, 4870.295350546845]},
 'dist': {'R2': [0.16943407382491632, 0.18401755037351225],
  'RMSE': [649.3841484106034, 682.964131050179],
  'intercept': [3697.38996191378, 4870.295350546845]},
 'hour': {'R2': [0.16943407382491632, 0.18401755037351225],
  'RMSE': [649.3841484106034, 682.964131050179],
  'intercept': [3697.38996191378, 4870.295350546845]},
 'birthyear': {'R2': [0.16943407382491632, 0.18401755037351225],
  'RMSE': [649.3841484106034, 682.964131050179],
  'intercept': [3697.38996191378, 4870.295350546845]},
 'usertype': {'R2': [0.16943407382491632, 0.18401755037351225],
  'RMSE': [649.3841484106034, 682.964131050179],
  'intercept': [3697.38996191378, 4870.295350546845]}}

### Mode Replacement

In [81]:
model_metrics_mode ={}
for var in features_for_amputation:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mode",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    if(var == 'gender'):
        for v in cat_gender:
            mu_train = X_train[v].mean()
            sigma_train = X_train[v].std()
            mu_test = X_test[v].mean()
            sigma_test = X_test[v].std()
            model_metrics_mode[v] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                            "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}
    elif(var == 'usertype'):
        for v in cat_usertype:
            mu_train = X_train[v].mean()
            sigma_train = X_train[v].std()
            mu_test = X_test[v].mean()
            sigma_test = X_test[v].std()
            model_metrics_mode[v] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                            "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}
    else:
        mu_train = X_train[var].mean()
        sigma_train = X_train[var].std()
        mu_test = X_test[var].mean()
        sigma_test = X_test[var].std()
        model_metrics_mode[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                              "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                                "intercept":model_reg.intercept_,
                              "Mu_train":mu_train, "sigma_train":sigma_train,
                              "Mu_test":mu_test, "sigma_test":sigma_test}
    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  56486
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  14122
Simulate Imputation for Mode
Train after Mode Imputation (282432, 5) (282432,)
Test after Mode Imputation (70608, 5) (70608,)
['gender', 'usertype']
2023-02-19 14:20:30.435282
2023-02-19 14:20:30.499878
Now model is running for varaible ==== dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  56486
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  14122
Simulate Imputation for Mode
Train after Mode Imputation (282432, 5) (282432,)
Test after Mode Imputation (70608, 5) (70608,)
['gender', 'usertype']
2023-02-19 14:20:32.007019
2023-02-19 14:20:32.087177
Now model is running for varaible ==== hour
Train (282432, 5

In [82]:
model_metrics_mode

{'gender_female': {'MAE': 278.7321056006803,
  'MSE': 429275.83119951247,
  'RMSE': 655.1914462197385,
  'R2': 0.16717760784514024,
  'intercept': 4526.713198285251,
  'Mu_train': 0.17032064355313845,
  'sigma_train': 0.37591491360780954,
  'Mu_test': 0.1679979605710401,
  'sigma_test': 0.3738671226914444},
 'gender_male': {'MAE': 278.7321056006803,
  'MSE': 429275.83119951247,
  'RMSE': 655.1914462197385,
  'R2': 0.16717760784514024,
  'intercept': 4526.713198285251,
  'Mu_train': 0.7817032064355314,
  'sigma_train': 0.4130906773077862,
  'Mu_test': 0.7845994788125992,
  'sigma_test': 0.41110282197329623},
 'gender_unknown': {'MAE': 278.7321056006803,
  'MSE': 429275.83119951247,
  'RMSE': 655.1914462197385,
  'R2': 0.16717760784514024,
  'intercept': 4526.713198285251,
  'Mu_train': 0.04797615001133016,
  'sigma_train': 0.213716168691913,
  'Mu_test': 0.04740256061636075,
  'sigma_test': 0.21249987622843455},
 'dist': {'MAE': 287.7887012385004,
  'MSE': 432003.53220551024,
  'RMSE': 

### Bootstrap with mode

In [83]:
bootstrap_data_mode = {}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mode",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx = np.arange(y_train.shape[0])
    bootstarp_train_accuracies = {}
    for i in range(501):
        train_idx = rng.choice(idx, size=idx.shape[0], replace=True)
        test_idx = np.setdiff1d(idx, train_idx, assume_unique=False)
        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]
        boot_test_X, boot_test_y = X_train.iloc[test_idx], y_train.iloc[test_idx]
        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_mode[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  56486
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  14122
Simulate Imputation for Mode
Train after Mode Imputation (282432, 5) (282432,)
Test after Mode Imputation (70608, 5) (70608,)
['gender', 'usertype']
Now running model is .... dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  56486
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  14122
Simulate Imputation for Mode
Train after Mode Imputation (282432, 5) (282432,)
Test after Mode Imputation (70608, 5) (70608,)
['gender', 'usertype']
Now running model is .... hour
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  56486
Test set after Amputatio

In [84]:
bootstrap_data_mode

{'gender': {'R2': [0.16973032735521126, 0.18328887417240441],
  'RMSE': [649.9661593370878, 682.3086117052005],
  'intercept': [3941.7687143969606, 5136.733952024837]},
 'dist': {'R2': [0.1650597668183234, 0.17807347049664235],
  'RMSE': [652.269048305782, 684.3071188015057],
  'intercept': [3430.0731945687094, 4544.887549308664]},
 'hour': {'R2': [0.16978808150000507, 0.1832118916726434],
  'RMSE': [649.9980485808713, 682.3219704229684],
  'intercept': [3694.4669898510856, 4824.266534073853]},
 'birthyear': {'R2': [0.17014831708532757, 0.18356607156988253],
  'RMSE': [649.7996569371485, 682.1833673320891],
  'intercept': [3047.6715971176936, 4143.338728569215]},
 'usertype': {'R2': [0.14569816237386468, 0.16030193995685388],
  'RMSE': [659.4080615971518, 692.6142244694461],
  'intercept': [3133.2711707980457, 4329.958767812104]}}

## Mean Imputation

In [85]:
variables =['dist','hour','birthyear']
model_metrics_mean ={}
for var in variables:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mean",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    model_metrics_mean[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                            "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}

    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  56486
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  14122
Simulate Imputation for Mean


C:\Users\Hinakoushar\AppData\Local\Temp\ipykernel_30400\1187816321.py:35: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_train.fillna(X_train.mean(), inplace=True)


Train after Mean Imputation (282432, 5) (282432,)


C:\Users\Hinakoushar\AppData\Local\Temp\ipykernel_30400\1187816321.py:37: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_test.fillna(X_test.mean(), inplace=True)


Test after Mean Imputation (70608, 5) (70608,)
['gender', 'usertype']
2023-02-19 14:30:20.507455
2023-02-19 14:30:20.584420
Now model is running for varaible ==== hour
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  56486
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  14122
Simulate Imputation for Mean


...

Test after Mean Imputation (70608, 5) (70608,)
['gender', 'usertype']
2023-02-19 14:33:27.533437
2023-02-19 14:33:27.604432
Now model is running for varaible ==== birthyear
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  56486
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  14122
Simulate Imputation for Mean


C:\Users\Hinakoushar\AppData\Local\Temp\ipykernel_30400\1187816321.py:35: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_train.fillna(X_train.mean(), inplace=True)


Train after Mean Imputation (282432, 5) (282432,)


C:\Users\Hinakoushar\AppData\Local\Temp\ipykernel_30400\1187816321.py:37: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_test.fillna(X_test.mean(), inplace=True)


Test after Mean Imputation (70608, 5) (70608,)
['gender', 'usertype']
2023-02-19 14:36:54.362092
2023-02-19 14:36:54.448855


In [86]:
model_metrics_mean

{'dist': {'MAE': 286.9939434102655,
  'MSE': 431890.3373680253,
  'RMSE': 657.1836405206883,
  'R2': 0.16210529973154308,
  'intercept': 3991.716784147813,
  'Mu_train': 0.9502712157262633,
  'sigma_train': 0.21738445116093827,
  'Mu_test': 0.9510112168592794,
  'sigma_test': 0.21584610744896857},
 'hour': {'MAE': 278.7067605557872,
  'MSE': 429076.69065292994,
  'RMSE': 655.0394573252286,
  'R2': 0.16756395316049721,
  'intercept': 4246.936417347307,
  'Mu_train': 0.9502712157262633,
  'sigma_train': 0.21738445116093827,
  'Mu_test': 0.9510112168592794,
  'sigma_test': 0.21584610744896857},
 'birthyear': {'MAE': 278.6542757636625,
  'MSE': 429039.20644985803,
  'RMSE': 655.0108445284384,
  'R2': 0.16763667489650347,
  'intercept': 4209.050052248725,
  'Mu_train': 0.9502712157262633,
  'sigma_train': 0.21738445116093827,
  'Mu_test': 0.9510112168592794,
  'sigma_test': 0.21584610744896857}}

### Bootstarp with mean

In [87]:
bootstrap_data_mean ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mean",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx = np.arange(y_train.shape[0])
    bootstarp_train_accuracies = {}
    for i in range(501):
        train_idx = rng.choice(idx, size=idx.shape[0], replace=True)
        test_idx = np.setdiff1d(idx, train_idx, assume_unique=False)
        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]
        boot_test_X, boot_test_y = X_train.iloc[test_idx], y_train.iloc[test_idx]
        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_mean[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  56486
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  14122
Simulate Imputation for Mean


C:\Users\Hinakoushar\AppData\Local\Temp\ipykernel_30400\1187816321.py:35: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_train.fillna(X_train.mean(), inplace=True)


Train after Mean Imputation (282432, 5) (282432,)


C:\Users\Hinakoushar\AppData\Local\Temp\ipykernel_30400\1187816321.py:37: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_test.fillna(X_test.mean(), inplace=True)


Test after Mean Imputation (70608, 5) (70608,)
['gender', 'usertype']
Now running model is .... dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  56486
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  14122
Simulate Imputation for Mean


...

Test after Mean Imputation (70608, 5) (70608,)
['gender', 'usertype']
Now running model is .... usertype
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  56486
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  14122
Simulate Imputation for Mean


C:\Users\Hinakoushar\AppData\Local\Temp\ipykernel_30400\1187816321.py:35: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_train.fillna(X_train.mean(), inplace=True)


Train after Mean Imputation (282432, 5) (282432,)


C:\Users\Hinakoushar\AppData\Local\Temp\ipykernel_30400\1187816321.py:37: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_test.fillna(X_test.mean(), inplace=True)


Test after Mean Imputation (70608, 5) (70608,)
['gender', 'usertype']


In [88]:
bootstrap_data_mean

{'gender': {'R2': [0.16989156190454985, 0.1833820125698682],
  'RMSE': [649.9049452488719, 682.2505426871476],
  'intercept': [3886.849201527024, 5071.303123311012]},
 'dist': {'R2': [0.16531885539751906, 0.17834603185987358],
  'RMSE': [652.1672695298977, 684.2169623184814],
  'intercept': [3433.16790862123, 4563.125048317238]},
 'hour': {'R2': [0.17000242512746255, 0.18335979779103656],
  'RMSE': [649.887471087138, 682.2235822377604],
  'intercept': [3702.1897027026453, 4834.145737227833]},
 'birthyear': {'R2': [0.17020072103125516, 0.18368555034055645],
  'RMSE': [649.7712867693974, 682.1602130928009],
  'intercept': [3549.819881578735, 4882.331160855227]},
 'usertype': {'R2': [0.14696456062111757, 0.16142359826544306],
  'RMSE': [658.9794378929798, 692.0545938809929],
  'intercept': [2715.711674710732, 3912.724502066233]}}

## Sampling Replacement Imputation

In [89]:
model_metrics_sample ={}
for var in features_for_amputation:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"sample",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    if(var == 'gender'):
        for v in cat_gender:
            mu_train = X_train[v].mean()
            sigma_train = X_train[v].std()
            mu_test = X_test[v].mean()
            sigma_test = X_test[v].std()
            model_metrics_sample[v] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                           "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}
    elif(var == 'usertype'):
        for v in cat_usertype:
            mu_train = X_train[v].mean()
            sigma_train = X_train[v].std()
            mu_test = X_test[v].mean()
            sigma_test = X_test[v].std()
            model_metrics_sample[v] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                           "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}
    else:
        mu_train = X_train[var].mean()
        sigma_train = X_train[var].std()
        mu_test = X_test[var].mean()
        sigma_test = X_test[var].std()
        model_metrics_sample[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                              "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                               "intercept":model_reg.intercept_,
                              "Mu_train":mu_train, "sigma_train":sigma_train,
                              "Mu_test":mu_test, "sigma_test":sigma_test}
    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  56486
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  14122
Simple Imputation using sampling
['gender', 'usertype']
2023-02-19 14:56:08.240108
2023-02-19 14:56:08.323575
Now model is running for varaible ==== dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  56486
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  14122
Simple Imputation using sampling
['gender', 'usertype']
2023-02-19 14:56:09.861408
2023-02-19 14:56:09.936337
Now model is running for varaible ==== hour
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  56486
Test set after Amputation (70608, 5)
X_test Amputed value count

In [90]:
model_metrics_sample

{'gender_female': {'MAE': 278.85396456035403,
  'MSE': 429384.3174478811,
  'RMSE': 655.2742307216736,
  'R2': 0.16696713762921922,
  'intercept': 4617.134205985364,
  'Mu_train': 0.21318405846363017,
  'sigma_train': 0.40955733369529207,
  'Mu_test': 0.20952300022660322,
  'sigma_test': 0.4069710779730033},
 'gender_male': {'MAE': 278.85396456035403,
  'MSE': 429384.3174478811,
  'RMSE': 655.2742307216736,
  'R2': 0.16696713762921922,
  'intercept': 4617.134205985364,
  'Mu_train': 0.7270387208248357,
  'sigma_train': 0.4454818985187002,
  'Mu_test': 0.7313193972354407,
  'sigma_test': 0.44327657207449467},
 'gender_unknown': {'MAE': 278.85396456035403,
  'MSE': 429384.3174478811,
  'RMSE': 655.2742307216736,
  'R2': 0.16696713762921922,
  'intercept': 4617.134205985364,
  'Mu_train': 0.0597772207115341,
  'sigma_train': 0.23707404665213513,
  'Mu_test': 0.05915760253795604,
  'sigma_test': 0.2359211073185802},
 'dist': {'MAE': 293.05216275275444,
  'MSE': 433936.9979399276,
  'RMSE':

### Bootstrap for sample

In [91]:
bootstrap_data_sample ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"sample",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx = np.arange(y_train.shape[0])
    bootstarp_train_accuracies = {}
    for i in range(501):
        train_idx = rng.choice(idx, size=idx.shape[0], replace=True)
        test_idx = np.setdiff1d(idx, train_idx, assume_unique=False)
        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]
        boot_test_X, boot_test_y = X_train.iloc[test_idx], y_train.iloc[test_idx]
        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_sample[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  56486
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  14122
Simple Imputation using sampling
['gender', 'usertype']
Now running model is .... dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  56486
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  14122
Simple Imputation using sampling
['gender', 'usertype']
Now running model is .... hour
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  56486
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  14122
Simple Imputation using sampling
['gender', 'usertype']
Now running model is .... birthyear
Train (282432, 5) (282432,)
Test (7

In [92]:
bootstrap_data_sample

{'gender': {'R2': [0.16963604782064934, 0.18305882195796896],
  'RMSE': [650.0546877990944, 682.3727173396347],
  'intercept': [4015.042044231823, 5226.601329635349]},
 'dist': {'R2': [0.16143660547137462, 0.17453452371209505],
  'RMSE': [653.7993057997849, 685.6542025352117],
  'intercept': [3236.8255879664525, 4360.930960177421]},
 'hour': {'R2': [0.1697306488068781, 0.18307582462188177],
  'RMSE': [650.0647525689808, 682.3269033940499],
  'intercept': [3684.2392817929895, 4817.60609230933]},
 'birthyear': {'R2': [0.17016139605304664, 0.18363719440102882],
  'RMSE': [649.7784402303014, 682.160494429482],
  'intercept': [3126.730810437811, 4295.344199071417]},
 'usertype': {'R2': [0.1324154670257659, 0.14525794900721967],
  'RMSE': [664.9092796507261, 698.0500296423436],
  'intercept': [2774.058005883161, 3939.267016260167]}}